In [4]:
import pandas as pd
user = pd.read_csv("target_users.tsv")
item = pd.read_csv("item_history.tsv",sep="\t")


In [2]:
item

,user_id,item_id,latest_timestamp,frequency
0,484,I355496,1587686400,1
1,484,I301315,1587686400,1
2,815,I146224,1583798400,1
3,998,I159337,1581033600,1
4,998,I69340,1580688000,1
...,...,...,...,...
6319941,1640781,I75214,1588032000,1
6319942,1640781,I103751,1588032000,1
6319943,1640781,I66854,1587772800,1
6319944,1640781,I102695,1588032000,1


In [5]:
target_user = pd.read_csv("target_users.tsv")
target_user.shape,len(set(target_user.user_id).intersection(item.user_id))

((250343, 1), 223265)

In [4]:
# user_master = pd.read_csv("user_master.tsv",sep="\t")

In [44]:
len(set(target_user.user_id) - set(item.user_id))

27078

In [4]:
item_ids = item.item_id.unique()
user_ids = item.user_id.unique()
item2idx = {item:idx for idx,item in enumerate(item_ids)}
idx2item = {idx:item for item,idx in item2idx.items()}
user2idx = {user:idx for idx,user in enumerate(user_ids)}
idx2user = {idx:user for idx,user in enumerate(user_ids)}


In [5]:
len(user_ids)

913707

In [5]:
item.item_id.max(),len(item.item_id.unique())

('I99999', 328823)

In [12]:
item = item.sort_values(by=['user_id', 'latest_timestamp'])
item_last = item.groupby("user_id").tail(1)
def hist_func(user_df):
    if len(user_df) == 1:
        return user_df
    else:
        return user_df[:-1]
from tqdm import tqdm
tqdm.pandas()

# item_hist = item.groupby('user_id').progress_apply(hist_func).reset_index(drop=True)

In [15]:
item_hist = pd.read_csv("item_hist.csv",index_col=None)

In [7]:

item_hist.head()

,user_id,item_id,latest_timestamp,frequency
0,2,I89717,1581897600,1
1,3,I311781,1585872000,1
2,3,I286302,1585872000,1
3,3,I304391,1585872000,1
4,3,I354326,1585872000,1


In [9]:
item_hist_last = item_hist.merge(item_last,on="user_id")

In [10]:
len(item_hist_last[item_hist_last.item_id_x==item_hist_last.item_id_y]), \
    len(item_hist_last), len(item_last)

(118837, 5525076, 913707)

In [16]:
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('latest_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['item_id'], df['latest_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')[['item_id', 'latest_timestamp']].progress_apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict
user_item_time_dict_hist = get_user_item_time(item_hist)
user_item_time_dict = get_user_item_time(item)

100%|██████████| 913707/913707 [01:59<00:00, 7629.77it/s]


In [2]:
import numpy as np
from collections import defaultdict
import math
from tqdm import tqdm
import pickle
def itemcf_sim(user_item_dict):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # user_item_time = get_user_item_time(df)
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                # print((0.99**(abs(i_click_time-j_click_time)/1000)))
                # i2i_sim[i][j] += (0.99**(abs(i_click_time-j_click_time)/1000)) \
                #     / math.log(len(item_time_list) + 1)
                click_time_weight = np.exp(0.99996 ** np.abs(i_click_time-j_click_time))
                i2i_sim[i][j] += click_time_weight / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in tqdm(i2i_sim.items()):
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open("./" + 'itemcf_i2i_sim3.pkl', 'wb'))
    
    return i2i_sim_

In [10]:
# itemcf_i2i2 = itemcf_sim(user_item_time_dict_hist)
# itemcf_i2i2 = pickle.load(open("./" + 'itemcf_i2i_sim2.pkl',"rb"))

In [3]:
# itemcf_i2i = itemcf_sim(user_item_time_dict)
itemcf_i2i = pickle.load(open("./" + 'itemcf_i2i_sim2.pkl',"rb"))

In [8]:
len(itemcf_i2i)#,len(itemcf_i2i2),len(set(item_last.item_id)-set(item_hist.item_id))

328823

In [6]:
i2i_recall = dict()
for item_t in tqdm(itemcf_i2i.keys()):
    sim_items_t = itemcf_i2i[item_t]
    sim_items_ranked = [[iid, w] for iid,w in sorted(sim_items_t.items(), key=lambda x:x[1],reverse=True)]
    i2i_recall[item_t] = sim_items_ranked

100%|██████████| 328823/328823 [02:51<00:00, 1921.68it/s] 


In [16]:
# pickle.dump(i2i_recall2,open("./itemcf_i2i_recall2.pkl",'wb'))
i2i_recall2 = pickle.load(open("./itemcf_i2i_recall2.pkl",'rb'))

In [7]:
pickle.dump(i2i_recall,open("./itemcf_i2i_recall2.pkl",'wb'))
# i2i_recall = pickle.load(open("./itemcf_i2i_recall.pkl",'rb'))

MemoryError: 

In [9]:
len(i2i_recall)#, len(i2i_recall2)

328823

In [46]:
sorted(itemcf_i2i[item_ids[0]].items(),key=lambda x: x[1],reverse=True)[:5]

[('I351459', 0.03590439231070408),
 ('I354830', 0.03266028434142804),
 ('I304799', 0.028486735055696027),
 ('I352249', 0.02457197351617471),
 ('I355548', 0.023506197679175903)]

In [57]:
len(set(item_last.item_id.unique()) -set(itemcf_i2i2.keys())),\
    len(set(item_hist.item_id.unique()) -set(itemcf_i2i2.keys())),\
    len(set(item_last.item_id.unique()) -set(itemcf_i2i.keys())),\
    len(item_last.item_id.unique())

(16785, 0, 0, 127126)

In [119]:
len(itemcf_i2i[item.iloc[999].item_id])

12277

In [58]:
len(itemcf_i2i),len(itemcf_i2i2),len(item_ids)

(328823, 312038, 328823)

In [20]:
len(item_hist["item_id"].unique()), len(item_ids)

(312038, 328823)

In [42]:
# count how many set similarity eddges
item_count1 = 0
item_count2 = 0
i_set1 = set()
i_set2 = set()
for item_t in tqdm(item_ids):
    items_sim1 = itemcf_i2i[item_t]
    items_sim2 = itemcf_i2i2.get(item_t,{})
    item_count1 += len(items_sim1)
    item_count2 += len(items_sim2)
    i_set1.update(items_sim1.keys())
    i_set2.update(items_sim2.keys())
    # i_set1.update([x for x in items_sim1])
    # i_set2.update([x for x in items_sim2])
item_count1,item_count2,len(i_set1),len(i_set2)

  0%|          | 0/328823 [00:00<?, ?it/s]


NameError: name 'itemcf_i2i2' is not defined

In [70]:
# 
item_count1 = 0
item_count2 = 0
last_in_hist = 0
i_set1 = set()
i_set2 = set()
with tqdm(total=len(user_ids)) as pbar:
    for user in user_ids:
        item_time_all = user_item_time_dict[user]
        item_time_hist = user_item_time_dict_hist[user]
        # last_item = item_last[item_last.user_id==user].item_id.values
        # last_in_hist += last_item in [x[0] for x in item_time_hist]
        item_count1+=len(item_time_all)
        item_count2+=len(item_time_hist)
        # for ii in item_time_all:
        #     i_set1.add(ii[0])
        # for ii in item_time_hist:
        #     i_set2.add(ii[0])
        pbar.update(1)
        # pbar.set_postfix(last_in_hist=last_in_hist)
len(item_hist),item_count1,item_count2,len(i_set1),len(i_set2)

100%|██████████| 913707/913707 [00:11<00:00, 81769.61it/s] 


(5525076, 6319946, 5525076, 0, 0)

In [159]:
len(user_item_time_dict_hist.keys())

913707

In [25]:
user_ids,len(user_ids)

(array([    484,     815,     998, ..., 1640422, 1640554, 1640781],
       dtype=int64),
 913707)

In [10]:
import random
user_recall_items_dict = {}
sim_item_topk = 10
recall_item_num = 100
hit = 0
related_items = 0
recall_item_count = 0

with tqdm(total=len(user_item_time_dict_hist)) as pbar:
    users_t = list(user_item_time_dict_hist.keys())
    random.shuffle(users_t)
    # exist_count = 0
    hist_count = 0
    hist_hit = 0
    for user in users_t:
        item_rank = {}
        gt_item = item_last[item_last.user_id==user].item_id.values
        existed_items = []
        existed = False
        for loc, (i, click_time) in enumerate(user_item_time_dict_hist[user]):
            existed_items.append(i)
            # if i==gt_item: 
                # exist_count +=1
                # existed = True
                # continue
            # if i not in itemcf_i2i2: continue
            # for j, wij in sorted(itemcf_i2i[i].items()\
            #                      ,key=lambda x:x[1],reverse=True)[:100]:#sim_item_topk]:
            for j, wij in i2i_recall[i][:]:
                item_rank.setdefault(j,0)
                item_rank[j]+=wij
        # print("related items", len(item_rank))
        recall_items = [x[0] for x in sorted(item_rank.items(), key=lambda x: x[1], \
                              reverse=True)[0:0+recall_item_num]]
        unsorted_items = [x[0] for x in item_rank.items()][:recall_item_num]
        
        # print(len(unsorted_items),len(set(recall_items).intersection(set(unsorted_items))))
        # recall_items = [x[0] for x in item_rank.items()][:recall_item_num]
        # print(item_rank.items())
        recall_item_count+=recall_item_num
        related_items += len(item_rank)
        hist_count += len(user_item_time_dict_hist[user])
        hist_hit += len([x[0] for x in user_item_time_dict_hist[user] if x[0] in recall_items])
        # print(gt_item,recall_items)
        hit += ((not existed) and (gt_item in recall_items))
        pbar.update(1)
        pbar.set_postfix(hit_rate=hit/pbar.n,recalled=recall_item_count,\
            related=related_items,hist_count=hist_count,\
                hist_hit=hist_hit)

NameError: name 'user_item_time_dict_hist' is not defined

In [16]:
import random
user_recall_items_dict = {}
sim_item_topk = 10
recall_item_num = 72
hit = 0
related_items = 0
recall_item_count = 0

with tqdm(total=len(user_item_time_dict)) as pbar:
    users_t = list(user_item_time_dict.keys())
    random.shuffle(users_t)
    # exist_count = 0
    hist_count = 0
    hist_hit = 0
    for user in users_t:
        item_rank = {}
        gt_item = item_last[item_last.user_id==user].item_id.values
        existed_items = []
        existed = False
        for loc, (i, click_time) in enumerate(user_item_time_dict[user]):
            existed_items.append(i)
            # if i==gt_item: 
                # exist_count +=1
                # existed = True
                # continue
            # if i not in itemcf_i2i2: continue
            # for j, wij in sorted(itemcf_i2i[i].items()\
            #                      ,key=lambda x:x[1],reverse=True)[:100]:#sim_item_topk]:
            for j, wij in i2i_recall[i][:]:
                item_rank.setdefault(j,0)
                item_rank[j]+=wij
        # print("related items", len(item_rank))
        recall_items = [x[0] for x in sorted(item_rank.items(), key=lambda x: x[1], \
                              reverse=True)[0:0+recall_item_num]]
        unsorted_items = [x[0] for x in item_rank.items()][:recall_item_num]
        
        # print(len(unsorted_items),len(set(recall_items).intersection(set(unsorted_items))))
        # recall_items = [x[0] for x in item_rank.items()][:recall_item_num]
        # print(item_rank.items())
        recall_item_count+=recall_item_num
        related_items += len(item_rank)
        hist_count += len(user_item_time_dict_hist[user])
        hist_hit += len([x[0] for x in user_item_time_dict_hist[user] if x[0] in recall_items])
        # print(gt_item,recall_items)
        hit += ((not existed) and (gt_item in recall_items))
        pbar.update(1)
        pbar.set_postfix(hit_rate=hit/pbar.n,recalled=recall_item_count,\
            related=related_items,hist_count=hist_count,\
                hist_hit=hist_hit)

  6%|▌         | 53041/913707 [1:22:44<22:22:37, 10.68it/s, hist_count=318450, hist_hit=191999, hit_rate=0.477, recalled=3818952, related=1.62e+9]


MemoryError: 

In [25]:
import random
user_recall_items_dict = {}
sim_item_topk = 200
recall_item_num = 75
hit = 0
related_items = 0
recall_item_count = 0
test_user = target_user.user_id.values
ret = []
with tqdm(total=len(test_user)) as pbar:
    users_t = list(test_user)
    random.shuffle(users_t)
    exist_count = 0
    hist_count = 0
    hist_hit = 0
    user_without_recall = 0
    for user in users_t:
        item_rank = {}
        gt_item = item_last[item_last.user_id==user].item_id.values
        existed_items = []
        existed = False
        for loc, (i, click_time) in enumerate(user_item_time_dict.get(user,[])):
            existed_items.append(i)
            if i==gt_item: 
                exist_count +=1
                # existed = True
                # continue
            # if i not in itemcf_i2i2: continue
            
            # for j, wij in sorted(itemcf_i2i[i].items()\
            #                      ,key=lambda x:x[1],reverse=True)[:100]:#sim_item_topk]:
            for j, wij in i2i_recall[i][:sim_item_topk]:
                item_rank.setdefault(j,0)
                item_rank[j]+=wij
        # print("related items", len(item_rank))
        # recall_items = [x[0] for x in sorted(item_rank.items(), key=lambda x: x[1], \
        #                       reverse=True)[0:0+recall_item_num]]
        recall_items = [x[0] for x in sorted(item_rank.items(), key=lambda x: x[1], \
                              reverse=True)[0:0+recall_item_num]]
        unsorted_items = [x[0] for x in item_rank.items()][:recall_item_num]
        if len(recall_items)==0:
            user_without_recall+=1
        # print(len(unsorted_items),len(set(recall_items).intersection(set(unsorted_items))))
        # recall_items = [x[0] for x in item_rank.items()][:recall_item_num]
        # print(item_rank.items())
        recall_item_count+=recall_item_num
        related_items += len(item_rank)
        ret.append([user,recall_items])
        hist_count += len(user_item_time_dict_hist.get(user,[]))
        hist_hit += len([x[0] for x in user_item_time_dict_hist.get(user,[]) if x[0] in recall_items])
        # print(gt_item,recall_items)
        hit += ((not existed) and (gt_item in recall_items))
        pbar.update(1)
        pbar.set_postfix(hit_rate=hit/pbar.n,recalled=recall_item_count,\
            related=related_items,hist_count=hist_count,user_without_recall=user_without_recall,\
                hist_hit=hist_hit,exist_count=exist_count)

100%|██████████| 250343/250343 [29:22<00:00, 142.02it/s, exist_count=223265, hist_count=2246166, hist_hit=2019194, hit_rate=0.692, recalled=7.51e+8, related=3.78e+8, user_without_recall=27382]


In [24]:
 (gt_item in recall_items)

True

In [21]:
recall_items

['I227624',
 'I330559',
 'I31876',
 'I202820',
 'I285733',
 'I154254',
 'I89142',
 'I162225',
 'I308545',
 'I75653',
 'I233709',
 'I291698',
 'I66624',
 'I25609',
 'I80959',
 'I78879',
 'I159284',
 'I49715',
 'I181256',
 'I13214',
 'I300510',
 'I120336',
 'I353664',
 'I46213',
 'I87755',
 'I37567',
 'I31479',
 'I303610',
 'I102874',
 'I83189',
 'I43014',
 'I101533',
 'I191505',
 'I304442',
 'I311475',
 'I143499',
 'I132542',
 'I87376',
 'I33903',
 'I102695',
 'I74651',
 'I64542',
 'I85453',
 'I64190',
 'I135313',
 'I178461',
 'I8757',
 'I43430',
 'I75517',
 'I195243',
 'I62442',
 'I352834',
 'I254411',
 'I69814',
 'I107065',
 'I242166',
 'I23262',
 'I313206',
 'I7220',
 'I84787',
 'I66048',
 'I352517',
 'I73254',
 'I69637',
 'I88191',
 'I345830',
 'I80128',
 'I143887',
 'I38324',
 'I183655',
 'I87475',
 'I193883',
 'I108348',
 'I75130',
 'I51503',
 'I157912',
 'I147169',
 'I67467',
 'I324655',
 'I72103',
 'I56713',
 'I170746',
 'I221358',
 'I148792',
 'I65818',
 'I60174',
 'I17985',
 '

In [51]:
itemcf_i2i

KeyboardInterrupt: 

In [58]:
itemcf_i2i["I89717"]

{'I186989': 0.0030778581413267074,
 'I348924': 0.005896815522027699,
 'I129228': 0.003690060639634747,
 'I82097': 0.004075461327011195,
 'I120655': 0.009234091583116636,
 'I104988': 0.005361853298693617,
 'I311776': 0.02356462791902665,
 'I290347': 0.003946736009517557,
 'I330254': 0.003972367218182912,
 'I143983': 0.07249293989879392,
 'I336457': 0.008623272455730541,
 'I164579': 0.0069313154479376205,
 'I258686': 0.004364268665461216,
 'I239170': 0.003394344899719983,
 'I81038': 0.005261089978199388,
 'I198844': 0.004025176418913893,
 'I295284': 0.0029487766300794584,
 'I41077': 0.0030422976892617664,
 'I12024': 0.007757341110904006,
 'I304391': 0.002716565899362819,
 'I41981': 0.007159593273801884,
 'I276076': 0.024475770739596604,
 'I258714': 0.012009042187564076,
 'I29500': 0.006015678689344025,
 'I124645': 0.0012990390308433182,
 'I128322': 0.024475770739596604,
 'I252842': 0.0022124687708839387,
 'I46565': 0.0012789131100507927,
 'I102695': 0.004920973860663449,
 'I187121': 0.00

In [35]:
import numpy as np
ret_user = [x[0] for x in ret]
ret_items = [x[1][:72] for x in ret]
ret_df = pd.DataFrame({"user_id":ret_user, "recalled_item":ret_items})

In [36]:
ret_df.to_csv("item_cf_ret2.csv",index=None)

In [164]:
i,gt_item,recall_items

('I243468',
 array(['I101173'], dtype=object),
 ['I101685',
  'I103870',
  'I103123',
  'I106779',
  'I104622',
  'I79800',
  'I210052',
  'I64419',
  'I89142',
  'I64585',
  'I99909',
  'I286248',
  'I195458',
  'I67227',
  'I106270',
  'I112636',
  'I46072',
  'I31876',
  'I106251',
  'I90264',
  'I18259',
  'I108486',
  'I107839',
  'I111186',
  'I72103',
  'I104352',
  'I96588',
  'I81038',
  'I77219',
  'I212866',
  'I32705',
  'I273381',
  'I228444',
  'I124198',
  'I222329',
  'I108605',
  'I111745',
  'I93084',
  'I79087',
  'I117545',
  'I88916',
  'I111478',
  'I80865',
  'I89467',
  'I305072',
  'I346767',
  'I25609',
  'I130472',
  'I311776',
  'I144635'])

In [ ]:
import numpy as np
i2i_mat = np.zeros([len(itemcf_i2i),len(itemcf_i2i)],dtype=np.int8)


for k in tqdm(itemcf_i2i.keys()):
    for v in itemcf_i2i[k].keys():
        i2i_mat[item2idx[k],item2idx[v]] = itemcf_i2i[k][v]
        i2i_mat[item2idx[v],item2idx[k]] = itemcf_i2i[k][v]

MemoryError: Unable to allocate 101. GiB for an array with shape (328823, 328823) and data type int8

In [ ]:
recall_item_num = 20
hit = 0
with tqdm(total=len(item_hist['user_id'].unique())) as pbar:
    for user in item_hist['user_id'].unique():
        user_hist_items = [x[0] for x in user_item_time_dict[user]]
        # print(user_hist_items)
        user_hist_item_ids = [item2idx[x] for x in user_hist_items]
        item_sims = np.sum(i2i_mat[user_hist_item_ids],axis=0)
        top_item_ids = np.argsort(item_sims)[-recall_item_num:]
        top_items = [idx2item[x] for x in top_item_ids]
        # user_recall_items_dict[user] = top_items
        pbar.update(1)
        item_gt = item_last[item_last.user_id==user].item_id.values
        # print(item_gt,top_items)
        hit+= (item_gt in top_items)
        pbar.set_postfix(loss=hit)

In [166]:
'I70197' in \
    ['I68683', 'I72457', 'I75255', 'I85802', 'I261183', 'I74918', 'I82860', 'I265341', 'I98924', 'I330923', 'I110613', 'I323398', 'I133078', 'I339673', 'I66854', 'I287811', 'I148655', 'I46072', 'I102695', 'I77219', 'I196033', 'I222602', 'I75517', 'I280182', 'I213495', 'I68734', 'I348943', 'I126086', 'I93641', 'I348207', 'I84787', 'I31876', 'I72579', 'I81491', 'I120341', 'I91547', 'I79941', 'I256937', 'I82249', 'I177254', 'I81038', 'I225744', 'I85126', 'I98788', 'I77172', 'I111186', 'I41039', 'I78276', 'I107176', 'I207398']


False

In [50]:
item_last[item_last.user_id==2].item_id.values

array(['I186989'], dtype=object)

In [62]:
import numpy as np
if np.array(['I46025']) in  ['I46025', 'I166374', 'I23392', 'I75508', 'I159503', 'I283257', 'I329198', 'I5146', 'I99576', 'I85487']:
    print("fjdskalfd")

fjdskalfd
